In [5]:
import numpy as np
from sklearn.decomposition import PCA
from tqdm import tqdm
from sklearn.model_selection import train_test_split
from torch.utils.data import Dataset, DataLoader
import torch
from torch import nn as nn
import pickle


In [6]:
class NumpyDataset(Dataset):
    def __init__(self, data):
        self.data = data
        
    def __len__(self):
        return len(self.data)
    
    def __getitem__(self, idx):
        return torch.tensor(self.data[idx, :-1], dtype=torch.float), torch.tensor(self.data[idx, -1], dtype=torch.float)

def create_dataloaders_from_df(dataframe, test_size=0.2, batch_size=32, shuffle=True):
    # Split the DataFrame into training and testing sets
    train_df, test_df = train_test_split(dataframe, test_size=test_size)
    
    # Create datasets
    train_dataset = NumpyDataset(train_df)
    test_dataset = NumpyDataset(test_df)
    
    # Create dataloaders
    train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)
    test_dataloader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    
    return train_dataloader, test_dataloader

class Classifier(nn.Module):
    def __init__(self, vec_dim, hidden_dim1, hidden_dim2, dropout_rate=0.1):
        super(Classifier, self).__init__()
        self.fc1 = nn.Linear(vec_dim, hidden_dim1)
        self.dropout1 = nn.Dropout(dropout_rate)
        self.fc2 = nn.Linear(hidden_dim1, hidden_dim2)
        self.dropout2 = nn.Dropout(dropout_rate)
        self.fc3 = nn.Linear(hidden_dim2, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()
        
    def forward(self, x):
        x = self.relu(self.fc1(x))
        #x = self.dropout1(x)
        x = self.relu(self.fc2(x))
        #x = self.dropout2(x)
        x = self.fc3(x)
        x = self.sigmoid(x)
        return x
    
def train_model(model, train_loader, test_loader, n_epochs, lr=0.001):
    accs = []
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    optimizer=torch.optim.Adam(model.parameters(), lr=lr)
    loss_fn=nn.BCEWithLogitsLoss()
    for epoch in range(n_epochs):
        model.train()
        total_loss = 0
        for batch in tqdm(train_loader):
            inputs, labels = batch
            inputs, labels = inputs.to(device), labels.to(device)
            
            optimizer.zero_grad()  
            outputs = model(inputs) 
            loss = loss_fn(outputs.squeeze(), labels)  
            loss.backward() 
            optimizer.step()  
            total_loss += loss.item()
        print(f'Loss: {total_loss:.2f}')
        acc = evaluate_model(model, test_loader)
        print(f'Testing accuracy: {acc}')
        accs.append(acc)
    return accs
        
        
def evaluate_model(model, test_loader):
    model.eval() 
    correct = 0
    total = 0
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    with torch.no_grad():  
        for inputs, labels in test_loader:
            inputs = inputs.to(device)
            labels = labels.to(device)
            outputs = model(inputs)
            predicted = (outputs.squeeze() >= 0.5).float()
            total += labels.size(0)
            correct += (predicted == labels).sum().item()
    
    accuracy = correct / total
    return accuracy

def append_number_to_rows(arr, num):
    num_column = np.full((arr.shape[0], 1), num)
    return np.hstack((arr, num_column))

In [7]:
n_components = [100]
accuracies = []
for n in n_components:
    print(f"Training model with {n} components")
    with open('ebt_full.pickle', 'rb') as file:
        data = pickle.load(file)
    print(data.shape)
    pca = PCA(n_components=n)
    pca.fit(data)
    low_dim_data = pca.transform(data)
    data = 0
    low_dim_neg_lbl = append_number_to_rows(low_dim_data[:1250000,:], 0)[:, :]
    low_dim_pos_lbl = append_number_to_rows(low_dim_data[1250000:,:], 1)[:, :]
    low_dim_data = 0
    low_dim_complete_lbl = np.vstack((low_dim_neg_lbl, low_dim_pos_lbl))
    low_dim_neg_lbl = 0
    low_dim_pos_lbl = 0
    train_loader_low_dim, test_loader_low_dim = create_dataloaders_from_df(low_dim_complete_lbl)
    model_low_dim = Classifier(n, 20, 10)
    accs = train_model(model_low_dim, train_loader_low_dim, test_loader_low_dim, n_epochs=100, lr=0.001)
    accuracies.append(accs)

Training model with 100 components
(2500000, 2292)


100%|███████████████████████████████████████████████████████████████████████████| 62500/62500 [00:38<00:00, 1626.54it/s]


Loss: 36806.29
Testing accuracy: 0.821776


100%|███████████████████████████████████████████████████████████████████████████| 62500/62500 [00:38<00:00, 1631.58it/s]


Loss: 36629.01
Testing accuracy: 0.824248


...

Loss: 36402.94
Testing accuracy: 0.825164


100%|███████████████████████████████████████████████████████████████████████████| 62500/62500 [00:37<00:00, 1666.58it/s]


Loss: 36402.80
Testing accuracy: 0.827858


100%|███████████████████████████████████████████████████████████████████████████| 62500/62500 [00:38<00:00, 1644.64it/s]


Loss: 36401.86
Testing accuracy: 0.827294


In [8]:
#np.savetxt('accuracies2.txt', accuracies)

In [9]:
np.savetxt('accuracies100epochs.txt', accuracies)